In [35]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Intro") \
    .getOrCreate()

## Map and Reduce

We've talked about these concepts in the intro_python module, but we didn't talk explicity about reductions.

MAP

```python
def squarer(a):
    return a**2
```

REDUCE

The prototypical reduce function is the sum function. A reducer takes two values: the first value is called an "accumulator" and the second value is the next value in the list. The goal of the reduce function is to transform the iterable into a new object that is "accumulated" as one iterates through the items in the iterable:

```python
def summer(a,b):
    return a + b
```

In [2]:
import pandas as pd

orders = pd.read_csv('data/orders.csv').to_dict(orient='records')

In [3]:
# We will start with the basic reduce function in Python
# NOTE: we have previously been using for comprehensions in python for mapping, 
# but there is also a built-in "map" function
from functools import reduce

In [11]:
def make_total_sale(order):
    return order['quantity_ordered'] * order['price_each']

# What does this return???
map(make_total_sale, orders)
print(make_total_sale(orders[0]))

4080.0


In [14]:
def summer(a,b):
    return a + b

# Get the total sales!
reduce(summer, map(make_total_sale, orders))

9604190.609999985

In [19]:
# Note, reduce takes an optional "initial value":

# See what happens when you change it from 0!
reduce(summer, map(make_total_sale, orders), 0.0)

9604190.609999985

In [31]:
# Challenge: make a "map" and a "reduce" that gets the max number of items ordered (in one line item)!

def max_orders(oln):
    return max(oln)

def len_order_line_number(order):
    return len(order['order_line_number'])

reduce(max_orders, map(len_order_line_number, orders), 0.0)
# Your code here

TypeError: object of type 'Column' has no len()

In [7]:
# Sometimes our data comes in JSON and it might be nested. Take a look at this format: 

import json

with open('data/orders.json') as f:
    orders = [json.loads(l) for l in f]

orders

[{'line_items': [{'product_code': 'S18_1749',
    'quantity_ordered': 30,
    'price_each': 136.0,
    'order_line_number': 3},
   {'product_code': 'S18_2248',
    'quantity_ordered': 50,
    'price_each': 55.09,
    'order_line_number': 2},
   {'product_code': 'S18_4409',
    'quantity_ordered': 22,
    'price_each': 75.46,
    'order_line_number': 4},
   {'product_code': 'S24_3969',
    'quantity_ordered': 49,
    'price_each': 35.29,
    'order_line_number': 1}],
  'order_number': 10100,
  'order_date': '2003-01-06',
  'required_date': '2003-01-13',
  'shipped_date': '2003-01-10',
  'status': 'Shipped',
  'comments': None,
  'sales_rep_employee_number': 1216,
  'customer': {'customer_number': 363,
   'customer_name': 'Online Diecast Creations Co.',
   'contact_last_name': 'Young',
   'contact_first_name': 'Dorothy',
   'country': 'USA',
   'city': 'Nashua',
   'state': 'NH',
   'credit_limit': 114200.0}},
 {'line_items': [{'product_code': 'S18_2325',
    'quantity_ordered': 25,
    

In [ ]:
# Challenge: Find the sum of the total number of sales with this new format.
# Everything should be accomplished in a map and then a reduce:

# Your code here

In [25]:
# Spark lets us read JSON files and create dataframes with nested items!

orders = spark.read.json('data/orders.json')

orders.printSchema()

root
 |-- comments: string (nullable = true)
 |-- customer: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- contact_first_name: string (nullable = true)
 |    |-- contact_last_name: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- credit_limit: double (nullable = true)
 |    |-- customer_name: string (nullable = true)
 |    |-- customer_number: long (nullable = true)
 |    |-- state: string (nullable = true)
 |-- line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- order_line_number: long (nullable = true)
 |    |    |-- price_each: double (nullable = true)
 |    |    |-- product_code: string (nullable = true)
 |    |    |-- quantity_ordered: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_number: long (nullable = true)
 |-- required_date: string (nullable = true)
 |-- sales_rep_employee_number: long (nullable = true)
 |-- shipped_date: string (nullable = 

In [26]:
# All "DataFrames" in Spark are backed by an RDD. 
# We can access the RDD to perform simple operations, just like in python,
# on the underlying data:

# Spark performs all operations lazily. "collect" tells it to gather all the items
# in a list and send it back to the Driver node. 
# If you try and collect too much data, your memory will blow up!

orders.rdd.filter(lambda r: r.customer.country == 'Belgium').collect()

[Row(comments=None, customer=Row(city='Charleroi', contact_first_name='Pascale ', contact_last_name='Cartrain', country='Belgium', credit_limit=23500.0, customer_name='Royale Belge', customer_number=381, state=None), line_items=[Row(order_line_number=1, price_each=60.28, product_code='S32_3207', quantity_ordered=27)], order_date='2003-04-11', order_number=10116, required_date='2003-04-19', sales_rep_employee_number=1401, shipped_date='2003-04-13', status='Shipped'),
 Row(comments=None, customer=Row(city='Charleroi', contact_first_name='Pascale ', contact_last_name='Cartrain', country='Belgium', credit_limit=23500.0, customer_name='Royale Belge', customer_number=381, state=None), line_items=[Row(order_line_number=1, price_each=56.41, product_code='S32_4289', quantity_ordered=20)], order_date='2003-08-13', order_number=10144, required_date='2003-08-21', sales_rep_employee_number=1401, shipped_date='2003-08-14', status='Shipped'),
 Row(comments=None, customer=Row(city='Bruxelles', contact

In [37]:
# Challenge

# Let's try and repeat the previous operation, of finding the total sales, 
# but now with the spark RDD. 
# You will need to read the Spark Documentation and find the following functions: map, reduce
# which are available as methods directly on the RDD!
# play around: 

orders.take(1)[0]
# your code
from operator import add

orders.rdd \
    .flatMap(lambda o: o.line_items)\
    .map(lambda o: o.price_each * o.quantity_ordered)\
    .reduce(add)

9604190.609999985

In [ ]:
# Spark thinks about where the Data lives. 
# RDD's have a concept of "key, value"
# This is implemented simply as a Tuple: (k,v)

# If we create an RDD that takes that form, a tuple of (k,v)
# we can use operations like "reduceByKey"!


# Challenge

# Get the total sales by country, by first mapping your RDD into a tuple (k,v) where
# the Key is the country, then reducing by summing the total sales. 

## Parquet

Parquet is another data format that plays well with Spark. 
 
It's a "flat file" format, like JSON or CSV, but it contains extra information about types, allows for "predicate pushdown", is column-oriented, and has first-class support for nested columns!

Predicate pushdown means that Spark doesn't have to read all the data from the disk! It can avoid certain sections of disk altogether because Parquet knows that we don't want that data.

In [42]:
orders = spark.read.parquet('data/orders')

In [ ]:
# Here, Spark won't read any information about countries other than Belgium!
# NOTE: the nested type!

orders.createOrReplaceTempView('orders')

res = spark.sql("""
SELECT count(order_number)
FROM orders 
WHERE customer.country = 'Belgium'
""".strip())

res.show()

## Nested types in SQL!

How do we deal with these pesky nested types now? 

Spark SQL gives us the Map/Reduce formulas to deal with nested "Array" types!

map = TRANSFORM
reduce = AGGREGATE

You can look at the documentation to see exactly how they work. 

In [ ]:
# Bonus Challenge: 

# Try to reproduce what we did before, getting the total sales, in Spark SQL, using TRANSFORM and AGGREGATE on the individuals "line_items" and then summing over the rows to get the total sales